In [ ]:
# second notebook for Yelp1 Labs 18 Project
# data cleanup

In [27]:
# imports

# dataframe
import pandas as pd
import json

# NLP
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

In [2]:
# import review.json file from https://www.yelp.com/dataset
with open('/Users/ianforrest/Desktop/coding/repos/yelp/yelp_dataset/review.json') as f:
    review = json.loads("[" + 
        f.read().replace("}\n{", "},\n{") + 
    "]")

In [3]:
# convert review.json files to pandas DataFrame 'df_review'
df_review = pd.DataFrame(review)

In [4]:
# check df_review to make sure it was created correctly
df_review.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,6,1,0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5.0,0,0,0,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33
2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5.0,3,0,0,I have to say that this office really has it t...,2016-11-09 20:09:03
3,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5.0,0,0,0,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38
4,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1.0,7,0,0,Today was my second out of three sessions I ha...,2018-01-30 23:07:38


In [5]:
# check column names of df_review
df_review.columns

Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'text', 'date'],
      dtype='object')

In [6]:
# check value counts of 'stars' column
df_review['stars'].value_counts()

5.0    2933082
4.0    1468985
1.0    1002159
3.0     739280
2.0     542394
Name: stars, dtype: int64

In [7]:
# check value counts of useful column
df_review['useful'].value_counts()

 0      3570453
 1      1430632
 2       669040
 3       349808
 4       201772
         ...   
 239          1
 235          1
 232          1
 229          1
-1            1
Name: useful, Length: 289, dtype: int64

In [8]:
# check value counts of funny column
df_review['funny'].value_counts()

0      5312173
1       813583
2       255974
3       111881
4        59525
        ...   
231          1
237          1
239          1
241          1
116          1
Name: funny, Length: 231, dtype: int64

In [9]:
# check value counts of cool column
df_review['cool'].value_counts()

 0      5013287
 1      1013484
 2       313433
 3       129433
 4        65993
         ...   
 179          1
 182          1
 183          1
 184          1
-1            1
Name: cool, Length: 201, dtype: int64

In [10]:
# check text of random reviews in dataset as part of initial exploration
df_review.iloc[3244,7]

'Would never return.  Do you think these guys are honest? We\'ve just moved to Las Vegas!!  Loyal BMW customers....I think we have had 10 BMWs altogether.  \n\nI walk into this dealership for a pre-inspection report on husband\'s 528 lease which is up.  The portfolio manager and salesguy inspect and tell me "its in great shape except for one ding and 2 tires" No paperwork but they say "Trust me - we will email you the inspection report--and don\'t worry! We have BMW financial $$ and you will not owe anything if you turn in your car here to us!"  \n\nFast forward =3 days:  I\'ve spoken to sales again and they promise they will, but they do not send my inspection report and now they will not answer voicemails. (nor will portfolio manager or general manager) \n\nI guess we will just order our next BMW from another dealer who actually does want our business.'

In [11]:
# check text of random reviews in dataset as part of initial exploration
df_review.iloc[2342553,7]

'Not worth the hype. It\'s kinda cute how they stick to the mob/speakeasy vibe. Also the staff is very kind. That\'s where the positives end, unfortunately. \n\nThe decor is old school, but not in a good way. Instead of looking like a place that was inspired by decades past, it just looks like the decor hasn\'t been updated in decades. \n\nThe food is nothing to write home about, especially for the price. I started with the "bust a cap" stuffed mushrooms, and was not impressed. I was expecting to have my mind blown, but was instantly disappointed instead. Bland and flavorless from the first bite. For my main course, I had a "chef sleeps with the fishes". This was better without a doubt, but still only slightly above average. \n\nI felt gypped. I honestly could have had better "Italian" food at an Olive Garden for less than half the price. Or, alternatively, I could have gotten far superior Italian food on the Strip for a similar price point. \n\nAlso, I think it\'s worth mentioning tha

In [12]:
# check text of random reviews in dataset as part of initial exploration
df_review.iloc[3,7]

"Went in for a lunch. Steak sandwich was delicious, and the Caesar salad had an absolutely delicious dressing, with a perfect amount of dressing, and distributed perfectly across each leaf. I know I'm going on about the salad ... But it was perfect.\n\nDrink prices were pretty good.\n\nThe Server, Dawn, was friendly and accommodating. Very happy with her.\n\nIn summation, a great pub experience. Would go again!"

In [ ]:
# export df_review to .csv
#df_review.to_csv(r'/Users/ianforrest/Desktop/coding/repos/yelp/yelp_dataset/df_review.csv')

In [13]:
# create copy of dataframe to manipulate for model
df = df_review.copy()

In [14]:
df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,6,1,0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5.0,0,0,0,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33
2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5.0,3,0,0,I have to say that this office really has it t...,2016-11-09 20:09:03
3,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5.0,0,0,0,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38
4,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1.0,7,0,0,Today was my second out of three sessions I ha...,2018-01-30 23:07:38


In [16]:
# add 'total_votes' column to dataframe; total of 'useful', 'funny', 'cool' columns
df['total_votes'] = df['useful'] + df['funny'] + df['cool']

In [17]:
df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,total_votes
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,6,1,0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36,7
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5.0,0,0,0,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33,0
2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5.0,3,0,0,I have to say that this office really has it t...,2016-11-09 20:09:03,3
3,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5.0,0,0,0,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38,0
4,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1.0,7,0,0,Today was my second out of three sessions I ha...,2018-01-30 23:07:38,7


In [18]:
# drop unused columns from dataframe
df = df.drop(columns=['user_id', 'business_id', 'review_id', 'useful', 'funny', 'cool'])

In [33]:
df.head()

,stars,text,date,total_votes
0,1.0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36,7
1,5.0,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33,0
2,5.0,I have to say that this office really has it t...,2016-11-09 20:09:03,3
3,5.0,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38,0
4,1.0,Today was my second out of three sessions I ha...,2018-01-30 23:07:38,7


In [24]:
# convert 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'])

In [25]:
df.dtypes

stars                 float64
text                   object
date           datetime64[ns]
total_votes             int64
dtype: object

In [26]:
# check value counts of 'total_votes' column
df['total_votes'].value_counts()

 0      3172103
 1      1157537
 2       661232
 3       483831
 4       293820
         ...   
 412          1
 416          1
 423          1
 426          1
-2            1
Name: total_votes, Length: 491, dtype: int64

In [28]:
# limit dataframe to reviews with 0 or more total votes
df = df.loc[df['total_votes'] >= 0]

In [29]:
# check value counts of 'total_votes' column
df['total_votes'].value_counts()

0       3172103
1       1157537
2        661232
3        483831
4        293820
         ...   
408           1
412           1
416           1
423           1
2256          1
Name: total_votes, Length: 490, dtype: int64

In [37]:
# remove html code from text column
df['text'] = df['text'].str.replace('(\d{1,2}[/. ](?:\d{1,2}|January|Jan)[/. ]\d{2}(?:\d{2})?)', '')
df['text'] = df['text'].str.replace('\n\n', '')                                    
df['text'] = df['text'].str.replace('\\n', '')
df['text'] = df['text'].str.replace('\n', '')

In [38]:
# check text of random reviews in dataset to make sure HTML code is removed correctly
# backslashes before apostrophes are for display purposes only to indicate apostrophes are not quotation marks
df.iloc[2342553,1]

'Not worth the hype. It\'s kinda cute how they stick to the mob/speakeasy vibe. Also the staff is very kind. That\'s where the positives end, unfortunately. The decor is old school, but not in a good way. Instead of looking like a place that was inspired by decades past, it just looks like the decor hasn\'t been updated in decades. The food is nothing to write home about, especially for the price. I started with the "bust a cap" stuffed mushrooms, and was not impressed. I was expecting to have my mind blown, but was instantly disappointed instead. Bland and flavorless from the first bite. For my main course, I had a "chef sleeps with the fishes". This was better without a doubt, but still only slightly above average. I felt gypped. I honestly could have had better "Italian" food at an Olive Garden for less than half the price. Or, alternatively, I could have gotten far superior Italian food on the Strip for a similar price point. Also, I think it\'s worth mentioning that they charge $8

In [39]:
# initiate STOPWORDS for NLP Processing
STOPWORDS = set(STOPWORDS).union(set(['I', 'We', 'i', 'we', 'it', "it's", 
                                      'it', 'the', 'this', 'they', 'They',
                                      'he', 'He', 'she', 'She', '\n', '\n\n']))

In [46]:
# create tokenize function to tokenize review text
def tokenize(text):
    return [token for token in simple_preprocess(text, deacc=True, min_len=4, max_len=40) if token not in STOPWORDS]

In [48]:
# add tokens column to dataframe 
df['tokens'] = df['text'].apply(tokenize)

In [49]:
# check to make sure tokens were added to dataframe correctly
df.head()

,stars,text,date,total_votes,tokens
0,1.0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36,7,"[total, horrible, service, crooks, actually, n..."
1,5.0,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33,0,"[adore, travis, hard, rock, kelly, cardenas, s..."
2,5.0,I have to say that this office really has it t...,2016-11-09 20:09:03,3,"[office, organized, friendly, phillipp, great,..."
3,5.0,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38,0,"[went, lunch, steak, sandwich, delicious, caes..."
4,1.0,Today was my second out of three sessions I ha...,2018-01-30 23:07:38,7,"[today, second, sessions, paid, session, went,..."


In [50]:
# export cleaned dataframe with tokenized text to .csv file
df.to_csv(r'/Users/ianforrest/Desktop/coding/repos/yelp/yelp_dataset/df.csv')

In [54]:
df.sort_values(['total_votes'], ascending=False)

,stars,text,date,total_votes,tokens
1292098,1.0,"In retrospect, I should have known better than...",2010-08-01 16:04:17,2256,"[retrospect, known, better, step, foot, baking..."
1218724,2.0,Dinner for 1.- PrefaceI went to Amy's Baking C...,2014-04-21 05:03:59,1692,"[dinner, prefacei, went, baking, company, open..."
1319233,1.0,This restaurant is horrible. First off the blu...,2013-08-22 05:02:56,1423,"[restaurant, horrible, blue, ribbon, burger, s..."
1331304,5.0,Flew to Arizona a few months ago to try this. ...,2017-11-12 08:06:13,1353,"[flew, arizona, months, disappointed, place, c..."
1300630,3.0,I actually suspect that a lot of people who ha...,2013-09-15 19:22:59,1169,"[actually, suspect, people, written, reviews, ..."
...,...,...,...,...,...
1304281,5.0,I have been going to RG for about 3 years now....,2014-02-02 00:43:12,0,"[going, years, standard, changes, major, repai..."
1304279,5.0,My five stars speaks little for the quality of...,2014-06-09 00:36:00,0,"[stars, speaks, little, quality, food, lucky, ..."
3925755,5.0,"Friendly staff, comfortable fun place. Everyon...",2015-08-29 22:01:44,0,"[friendly, staff, comfortable, place, nice, ki..."
3925757,5.0,Passing through Monroeville and visiting famil...,2018-05-27 01:46:15,0,"[passing, monroeville, visiting, family, looki..."


In [55]:
df.iloc[1292098,1]

'In retrospect, I should have known better than to step foot into Amy\'s Baking Company (a.k.a. ABC Bistro).  -8 pm on a Saturday night, three tables are occupied-a sign on the door indicates they\'re looking for an experienced line cook, a dishwasher, waitresses, you name it-Pita Jungle, just a stone\'s throw away, is packed to the gillsThese are all bad signs.  They go ignored, however, because I\'d eaten one thing all day, and I had drove here to try their pizza.  Never, ever again.The waitress brings me out a water and a menu to my table for one  on the patio, which I had all to myself.  Browsing the apps, I was pleasantly surprised to see several things priced $3, which seemed like a bargain, given the high price of everything else.  $3 tapanade sounded like a good deal, but I was informed that it didn\'t come with anything - it was $3 extra for bread.  Since I was solo, I decided to skip it and instead pay $14 for their 12" margherita pizza.  About three minutes later, the waitre

In [1]:
df.dtypes

NameError: name 'df' is not defined